In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
# GOTO edit, notebook settings, GPU. 


Found GPU at: /device:GPU:0


In [ ]:
import os
import sys
import tensorflow as tf           # Training and inference of deep neural networks
import cv2                        # Computer Vision
import matplotlib.pyplot as plt   # Visualizations in Python
import numpy as np                # Calculating stuff

### Data Collection

In [ ]:
!mkdir database
!cd database

# Upload zip of dataset in google drive and make it open for everyone. Copy link
# Link looks like this: drive.google.com/file/d/someAlphaNumeric/view?usp=sharing, the someAlphaNumeric is your ID.

!gdown --id 1d7JR3352rqy-f6-fr6HGny7aNcMuXI2J

# Will be downloaded to working directory.

!unzip eye_database_4000x2.zip

# If not doing in colab then the method of uploading is left to your discretion.

In [ ]:
!ls 
training_data =[]                                                                      # Initialize list for dataset.
classes = ["closed", "open"]                                                           # 2 Classes; classes[0] = closed_eyes, classes[1] = open_eyes
datadirectory = r"."                                                                   # Directory of dataset, depends on how the user applies this. 
img_size = 224                                                                         # Let img_size that we defined be 224 pixels
def create_training_data():
    for category in classes:                                                           # For each category:
        path=os.path.join(datadirectory, category) 
        print(path)                                                                    # Prints path of file
        class_num= classes.index(category)
        for img in os.listdir(path):
            print(img)
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)   # Read image in GREYSCALE
                backtorgb = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)                 # Convert GRAY to RGB
                new_array = cv2.resize(backtorgb, (img_size,img_size))                 # Resize image to the img_size we set
                training_data.append((new_array, class_num))                           # Append new array and type (closed or open) to list of dataset
            except Exception as e:
                pass
create_training_data()                                                                 # run create_training_data() function.


In [ ]:
print(len(training_data)) # just checking xD

### Shuffling Data

In [ ]:
import random

random.shuffle(training_data)                                                          # Randomizing training_data list to prevent bias (if any)

x = []
y = []

for features,label in training_data:                                                   # Create new lists 
    x.append(features)                                                                 # x has image data
    y.append(label)                                                                    # y has label (open or closed) 

x = np.array(x).reshape(-1, img_size, img_size, 3 )                                    # np.array() converts argument into Class 'numpy.ndarray'
                                                                                       # reshape ndarray object into array of dimension of (-1, 224, 224, 3) (yes 4 dimensions here)
                                                                                       # Dimensions are counted in direction: from outside to inside. 
                                                                                       # [ [ [ [a, b, c], [d, e, f], ...244 times ], [ [a, b, c], [d, e, f], ...244 times ], ...244 times ] ...len(x) times]
                                                                                       # In this case dimension of -1 is equivalant to len(x).                                                                                     

In [ ]:
x.shape                                                                                # An attribute of ndarray. Prints tuple with each index having the number of elements in each dimension
x= x/255.0                                                                             # Normalizing each value with respect to it having 256 pixels
Y=np.array(y)

At `denomenator = (256-1)`, `x= x/255` might be too heavy.  
Try `denomenator = [(256/16)-1]` and keep going on to 32, 64...  
Then try using more datasets to get better accuracy.

### Pickling Data  
Pickle converts any kind of python objects (list, dict, etc.) into byte stream (0s and 1s). This is called pickling or serialization.

In [ ]:
import pickle 
pickle_out= open("x.pickle","wb")                                                     # Open pickle instance to write(w) in it in binary(b)
pickle.dump(x,pickle_out)                                                             # Dump list x into pickle
pickle_out.close()                                                                    # Create and save pickle instance 

pickle_out= open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()


pickle_in = open("x.pickle","rb")                                                     # Open pickle instance to read(r) in it in binary(b)
x = pickle.load(pickle_in)                                                            # Load information into variable x

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

### CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Yes it was imported again.

In [ ]:
model= tf.keras.applications.mobilenet.MobileNet()
model.summary()

Mobilenet is an architecture that uses depthwise separable convolutions to construct lightweight deep convolutional neural networks and provides an efficient model for mobile and embedded vision applications. It has fewer parameters and higher classification accuracy as compared to other architectures.  
By default, mobilenet uses softmax classifier.


### Transfer Learning

In [ ]:
base_input= model.layers[0].input
base_output = model.layers[-4].output

In [ ]:
Flat_layer= layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer)
final_output= layers.Activation('sigmoid')(final_output)

In [ ]:
new_model = keras.Model(inputs = base_input, outputs= final_output)
new_model.summary()

In [ ]:
new_model.compile(loss="binary_crossentropy", optimizer = "adam", metrics=["accuracy"])
new_model.fit(x,Y, epochs = 1,validation_split=0.1)

In [ ]:
new_model.save('my_model.h5')

In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')

### Checking Network for prediction

In [ ]:
img_array = cv2.imread(r'path/to/random_img', cv2.IMREAD_GRAYSCALE)
backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
new_array =cv2.resize(backtorgb, (img_size,img_size))

In [ ]:
x_input = np.array(new_array).reshape(1, img_size, img_size, 3)
x_input.shape

In [ ]:
plt.imshow(new_array)

In [ ]:
x_input=x_input/255.0

In [ ]:
prediction = new_model.predict(x_input)
prediction

## Predicting in unknown images


In [ ]:
img = cv2.imread(r'path/to/unknown_img')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
faceCascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

In [ ]:
eyes=eye_Cascade.detectMultiScale(gray,1.1,4)
for(x, y, w, h) in eyes[:]:
    cv2.rectangle(img, (x,y),(x+w, y+h), (0,255,0),2)
print(eyes)



In [ ]:
eye_Cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_eye.xml')
gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
eyes=eye_Cascade.detectMultiScale(gray,1.1,4)
# To identify eyes in unknown image
for(x, y, w, h) in eyes:
    roi_gray=gray[y:y+h, x:x+w]
    roi_color=img[y:y+h, x:x+w]
    eyess=eye_Cascade.detectMultiScale(roi_gray)
    if len(eyes) == 0:
            print("eyes are not detected")
    else:
        for(ex, ey, ew, eh) in eyess:
            eyes_roi = roi_color[ey: ey + eh, ex: ex + ew]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # Displays rectangle over detected eyes 

to_test

In [ ]:
plt.imshow(cv2.cvtColor(eyes_roi, cv2.COLOR_BGR2RGB)) # shows one identified eye

In [ ]:
eyes_roi.shape

In [ ]:
final_image=cv2.resize(eyes_roi,(224, 224))
final_image=np.expand_dims(final_image,axis=0)
final_image=final_image/225.0

final_image.shape

In [ ]:
new_model.predict(final_image)

## Applying to Webcam

In [ ]:
import cv2 

path  = "haarcascade frontalface_default.xml" 
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades+ 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly

if not cap.isOpened(): 
    cap = cv2.VideoCapture(1)
if not cap.isOpened(): 
    raise I0Error("Cannot open webcam")


while True:
    ret, frame = cap.read()
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    eyes = eye_Cascade.detectMultiScale(gray, 1.1, 4)

    for x,y,w,h in eyes:
        roi_gray = gray[y:y+h, x:x+w] 
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2) 
        eyess = eye_Cascade.detectMultiScale(roi_gray) 
        if len(eyes) == 0:
            print("Eyes not detected")
        else:
            for (ex,ey,ew,eh) in eyess: 
                eyes_roi = roi_color[ey: ey + eh, ex:ex + ew]
                
        final_image=cv2.resize(eyes_roi,(224, 224))
        final_image=np.expand_dims(final_image,axis=0)
        final_image=final_image/225.0



    Predictions = new_model.predict(final_image) 
    if (Predictions > 0):
        status = "Open_Eyes"
    else:
        status = "Closed Eyes"

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print(faceCascade.empty())
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    # To draw a rectangle around features
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Using putText() methhod for inserting text in video
    
    cv2.putText(frame, status, (50, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)
    cv2.imshow("Drowsiness Detected", frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()